## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [1]:
import os
import sys
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

# Adding the parent directory to the path to enable importing
root_dir = os.path.dirname(os.path.abspath("../"))
if root_dir not in sys.path:
    sys.path.append(root_dir)

import DDQN.DDQN as ddqn
from DDQN.DQN import TargetDQNAgent, DoubleDQNAgent
from DDQN.DDQN import DuelingDQNAgent, DoubleDuelingDQNAgent
from DDQN.evaluation import compare_agents, display_stats
from DDQN.trainer import RandomWeaknessBasicOpponent
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [2]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [3]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.0678153 , -0.06325722,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [4]:
agent_player = DoubleDuelingDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[512],
    hidden_sizes_A=[512, 512],
    hidden_sizes_V=[512, 512],
    use_torch=False
)

In [5]:
agent_player = DoubleDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[2048],
    use_torch=False
)

In [5]:
STATE_PATH = "./models/doub-duel-dqn-512/"

In [6]:
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)
agent_opp_random = RandomWeaknessBasicOpponent(0.2)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [7]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)
random_opp_stats = compare_agents(
    agent_player, agent_opp_random, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
print("#" * 40)
print("Stats Against Weak Opponent:")
display_stats(weak_opp_stats)

print()
print("#" * 40)
print("Stats Against Strong Opponent:")
display_stats(strong_opp_stats)

print()
print("#" * 40)
print("Stats Against Random Opponent:")
display_stats(random_opp_stats)

########################################
Stats Against Weak Opponent:
Player Win Rate: 0.135
Opponent Win Rate: 0.105
Draw Rate: 0.76

Win Status (1 for win, 0 for draw, -1 for loss):
  Mean: 0.03
  Std: 0.48897852713590606

Returns:
  Player: -2219.205389668123
  Opponent: -13131.250786889781
  Difference: 10912.045397221658

########################################
Stats Against Strong Opponent:
Player Win Rate: 0.142
Opponent Win Rate: 0.26
Draw Rate: 0.598

Win Status (1 for win, 0 for draw, -1 for loss):
  Mean: -0.118
  Std: 0.6229574624322274

Returns:
  Player: -4890.919107660536
  Opponent: -7369.00002899272
  Difference: 2478.0809213321845

########################################
Stats Against Random Opponent:
Player Win Rate: 0.123
Opponent Win Rate: 0.136
Draw Rate: 0.741

Win Status (1 for win, 0 for draw, -1 for loss):
  Mean: -0.013
  Std: 0.5087543611606685

Returns:
  Player: -2705.7121778556443
  Opponent: -12340.95905328861
  Difference: 9635.246875432966


### Rendered Demonstration Against Strong Opponent

In [14]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, info = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        winner = info["winner"]
        if winner == 0:
            print("Draw")
        else:
            print(f"Winner: " + ("Player" if winner == 1 else "Opponent"))
        break

Episode done in 211 steps
Winner: Opponent


In [15]:
env.close()